# API Face Samples

## Objective

To develop a Python-based application that detects and verifies faces within a face collection (face list or person group) using Azure Face SDK and REST API. The application will cover three scenarios:

- **Image + Face List**: Detect faces in the image and check if the largest detected face matches any face in the given face list.
- **Face ID + Face List**: Verify if the provided face ID matches any face in the given face list.
- **Image + Person Group**: Detect faces in the image and identify if the largest detected face belongs to any person in the given person group.
- **Face ID + Person Group**: Identify if the provided face ID belongs to any person in the given person group.

The function returns relevant identifiers and confidence scores, allowing users to determine the match accuracy and identity of faces in different contexts.


In [ ]:
import os, requests
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel

def find_faces_in_collection(subscription_key, endpoint, image_path=None, face_id=None, face_list_id=None, person_group_id=None):
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'x-ms-useragent': 'find-faces-in-collection/1.0.0'
    }
    
    def detect_face(image_path):
        with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"x-ms-useragent": "find-faces-in-collection/1.0.0"} ) as face_client:
            with open(image_path, 'rb') as image_data:
                detected_faces = face_client.detect(
                        image_content=image_data.read(),
                        detection_model=FaceDetectionModel.DETECTION_03,
                        recognition_model=FaceRecognitionModel.RECOGNITION_04,
                        return_face_id=True,
                    )
            return [face.face_id for face in detected_faces]

    def check_face_id_in_face_list(face_id, face_list_id):
        headers['Content-Type'] = 'application/json'
        find_similar_url = endpoint + '/face/v1.0/findsimilars'
        body = {
            'faceId': face_id,
            'faceListId': face_list_id,
            'maxNumOfCandidatesReturned': 1
        }
        response = requests.post(find_similar_url, headers=headers, json=body)
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"Error recognizing face in face list: {e}")
            print(f"Response: {response.text}")
            return {}
        similar_faces = response.json()
        
        if similar_faces and similar_faces[0]['confidence']:
            persisted_face_id = similar_faces[0]['persistedFaceId']
            confidence = similar_faces[0]['confidence']
            return {"persistedFaceId": persisted_face_id, "confidence": confidence}
        else:
            return {}

    def check_face_id_in_person_group(face_id, person_group_id):
        headers['Content-Type'] = 'application/json'
        identify_url = f"{endpoint}/face/v1.0/identify"
        body = {
            'personGroupId': person_group_id,
            'faceIds': [face_id],
            'maxNumOfCandidatesReturned': 1,
            'confidenceThreshold': 0.5
        }
        response = requests.post(identify_url, headers=headers, json=body)
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"Error recognizing face in person group: {e}")
            print(f"Response: {response.text}")
            return {}
        identify_results = response.json()
        if identify_results and identify_results[0]['candidates']:
            person_id = identify_results[0]['candidates'][0]['personId']
            confidence = identify_results[0]['candidates'][0]['confidence']
            return {"personId": person_id, "confidence": confidence}
        else:
            return {}
    
    if image_path:
        face_ids = detect_face(image_path)
        if not face_ids:
            raise ValueError("No face detected in the image.")
        else:
            face_id = face_ids[0]
    
    if face_list_id:
        return check_face_id_in_face_list(face_id, face_list_id)
    elif person_group_id:
        return check_face_id_in_person_group(face_id, person_group_id)
    else:
        raise ValueError("Either face_list_id or person_group_id must be provided.")

# Example usage:
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'
face_id = 'your_face_id'
face_list_id = 'your_face_list_id'
person_group_id = 'your_person_group_id'

# Image + Face List
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, image_path=image_path, face_list_id=face_list_id))

# Face ID + Face List
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, face_id=face_id, face_list_id=face_list_id))

# Image + Person Group
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, image_path=image_path, person_group_id=person_group_id))

# Face ID + Person Group
print(find_faces_in_collection(FACE_KEY, FACE_ENDPOINT, face_id=face_id, person_group_id=person_group_id))